In [1]:
import ktrain
from ktrain import text as txt
import pandas as pd

In [ ]:
train_small = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/train_small.csv")
test_small = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/test_small.csv")
train = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/train.csv")

In [ ]:
train.shape

In [ ]:
test_small.shape

In [ ]:
train_small.shape

In [ ]:
large_x_train, large_y_train = train['text'].tolist(), train['stars'].tolist()

In [ ]:
small_x_train, small_y_train = train_small['text'].tolist(), train_small['stars'].tolist()

In [ ]:
x_test, y_test = test_small['text'].tolist(), test_small['stars'].tolist()

In [ ]:
import pickle

In [ ]:
import boto3
from botocore.exceptions import ClientError

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
bucket = "yelp-dataset-pt-9"

In [ ]:
def preproc_and_save(large_or_small, x_train, y_train, x_test, y_test):
    trn, val, preproc = txt.texts_from_array(x_train=x_train, y_train=y_train,
                                          x_test=x_test, y_test=y_test,
                                          preprocess_mode='distilbert',
                                          maxlen=512)

    pickle.dump( trn, open( f"{large_or_small}/trn.p", "wb" ), protocol=4 )
    pickle.dump( val, open( f"{large_or_small}/val.p", "wb" ), protocol=4 )
    pickle.dump( preproc, open( f"{large_or_small}/preproc.p", "wb" ), protocol=4 )
    
    upload_file(f"{large_or_small}/trn.p", bucket, f'spencer/data/sentiment/en/ktrain/regression/long/{large_or_small}/trn.p')
    upload_file(f"{large_or_small}/val.p", bucket, f'spencer/data/sentiment/en/ktrain/regression/long/{large_or_small}/val.p')
    upload_file(f"{large_or_small}/preproc.p", bucket, f'spencer/data/sentiment/en/ktrain/regression/long/{large_or_small}/preproc.p')

In [ ]:
preproc_and_save("large", large_x_train, large_y_train, x_test, y_test)

In [ ]:
preproc_and_save("small", small_x_train, small_y_train, x_test, y_test)